In [7]:
# Importing our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Importing and read the charity_data.csv.
import pandas as pd 
new_application_df = pd.read_csv("Resources/charity_data.csv")
new_application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [8]:
# Dropping the non-beneficial ID columns, 'EIN' and 'NAME'.
new_application_df.drop(['EIN'], axis=1, inplace=True)
new_application_df.head(3)


,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [9]:
new_application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   NAME                    34299 non-null  object
 1   APPLICATION_TYPE        34299 non-null  object
 2   AFFILIATION             34299 non-null  object
 3   CLASSIFICATION          34299 non-null  object
 4   USE_CASE                34299 non-null  object
 5   ORGANIZATION            34299 non-null  object
 6   STATUS                  34299 non-null  int64 
 7   INCOME_AMT              34299 non-null  object
 8   SPECIAL_CONSIDERATIONS  34299 non-null  object
 9   ASK_AMT                 34299 non-null  int64 
 10  IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(8)
memory usage: 2.9+ MB


In [10]:
# Determining the number of unique values in each column.
unique_values_in_each_column = new_application_df.nunique()
unique_values_in_each_column

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [11]:
# Dropping additional non-beneficial columns, 'STATUS' and 'SPECIAL_CONSIDERATIONS'
new_application_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS'], axis=1, inplace=True)
new_application_df.head(3)

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0


In [12]:
# Looking at 'NAME' value counts for binning
name_value_counts = new_application_df['NAME'].value_counts()
name_value_counts

NAME
PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: count, Length: 19568, dtype: int64

In [14]:
# Looking for 'NAME" value counts greater than 4
name_value_counts[name_value_counts>4]

NAME
PARENT BOOSTER USA INC                                    1260
TOPS CLUB INC                                              765
UNITED STATES BOWLING CONGRESS INC                         700
WASHINGTON STATE UNIVERSITY                                492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC            408
                                                          ... 
ACTS MISSIONS                                                5
KAPPA ALPHA PSI FRATERNITY                                   5
SOCIETY OF MAYFLOWER DESCENDANTS IN THE STATE OF TEXAS       5
INTERNATIONAL CONGRESS OF CHURCHES AND MINISTRIES            5
PENN STATE ALUMNI ASSOCIATION                                5
Name: count, Length: 402, dtype: int64

In [15]:
# Determining which values can be replaced if counts are less than or equal to 4.
names_to_replace = list(name_value_counts[name_value_counts <= 4].index)

# Replacing values in the DataFrame
for name in names_to_replace:
  new_application_df['NAME'] = new_application_df['NAME'].replace(name, 'Other')
    
# Checking to make sure the binning process was successful
new_application_df['NAME'].value_counts()

NAME
Other                                                                19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: count, Length: 403, dtype: int64

In [16]:
# Looking at APPLICATION_TYPE value counts for binning
application_type_value_counts = new_application_df['APPLICATION_TYPE'].value_counts()
application_type_value_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [17]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

# Choosing a cutoff value for the number of occurrences
cutoff_value = 500 

# Creating a list of application types to be replaced
application_types_to_replace = application_type_value_counts[application_type_value_counts < cutoff_value].index.tolist()

# Replacing in DataFrame
for app in application_types_to_replace:
    new_application_df['APPLICATION_TYPE'] = new_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Checking to make sure binning was successful
new_application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [19]:
# Looking at CLASSIFICATION value counts for binning
classification_value_counts = new_application_df['CLASSIFICATION'].value_counts()
classification_value_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [20]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

# Choosing a cutoff value for the number of occurrences
cutoff_value = 1000  

# Create a list of classifications to be replaced
classifications_to_replace = classification_value_counts[classification_value_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    new_application_df['CLASSIFICATION'] = new_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
new_application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [23]:
# Generating an array of the DataFrames categorical variables
cat_variables = list(new_application_df.dtypes[new_application_df.dtypes == 'object'].index)
cat_variables

['NAME',
 'APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT']

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
new_application_df = pd.get_dummies(new_application_df)

# Displaying the first few rows of the updated DataFrame
new_application_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_AACE INTERNATIONAL,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,108590,1,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,5000,0,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,6692,1,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,142590,1,False,False,False,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [25]:
# Split our preprocessed data into our features and target arrays
y = new_application_df['IS_SUCCESSFUL'].values
X = new_application_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [26]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

# Defining the deep neural net model
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 30
hidden_nodes_layer3 = 10

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

c:\Users\larry\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 100)            │        44,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │         3,030 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,751 (186.53 KB)

 Trainable params: 47,751 (186.53 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
# Compiling the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Training the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 879us/step - accuracy: 0.7398 - loss: 0.5497
Epoch 2/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 841us/step - accuracy: 0.7952 - loss: 0.4469
Epoch 3/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.8033 - loss: 0.4274
Epoch 4/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 833us/step - accuracy: 0.7989 - loss: 0.4295
Epoch 5/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 858us/step - accuracy: 0.8030 - loss: 0.4217
Epoch 6/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 865us/step - accuracy: 0.8003 - loss: 0.4221
Epoch 7/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 855us/step - accuracy: 0.8032 - loss: 0.4192
Epoch 8/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step - accuracy: 0.8006 - loss: 0.4215
Epoch 9/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 866us/step - accuracy: 0.8010 - loss: 0.4168
Epoch 10/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 853us/step - accuracy: 0.8042 - loss: 0.4143
Epoch 11/200
804/804 ━━━━━━━━━━━━━━━━━━━━ 1s 850us/step - accuracy: 0.8041 - loss: 0.4148
Epoch 12/200
804/80

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - 930us/step - accuracy: 0.7886 - loss: 0.4557
Loss: 0.45570090413093567, Accuracy: 0.7885714173316956


In [33]:
# Export our model to HDF5 file using the native Keras format recommended by VSCode 

# Defining the filename for saving the model
filename = 'AlphabetSoupCharity_Optimization.keras'  

# Exporting the model to HDF5(native Keras format) file
nn_model.save(filename)

print(f"Model saved to {filename}")

Model saved to AlphabetSoupCharity_Optimization.keras
